# Laboratorio 1: Limpieza de Información Personal Identificable (PII) usando Expresiones Regulares

Una empresa de consultoría ha sido contratada por un hospital para limpiar registros de transcripciones de llamadas, eliminando toda información personal identificable para cumplir con regulaciones de privacidad y protección de datos.

### Objetivo
Desarrollar un sistema de ofuscación basado en regex que identifique y redacte los siguientes tipos de PII:
- **Correos electrónicos**
- **Números de teléfono** 
- **Números de identificación**
- **Direcciones**
- **Nombres propios**

### Metodología
Se utilizará únicamente la librería `re` de Python para crear patrones de expresiones regulares que detecten y reemplacen la información sensible.

### Repositorio
https://github.com/XJ4v1erX/NLP-L1-Expresiones-regulares 

## 1. Importación de Librerías

In [1]:
import pandas as pd
import re

## 2. Carga y Exploración de Datos

In [2]:
df = pd.read_csv('pii_dataset.csv')

## 3. Definición de Patrones de Expresiones Regulares

A continuación se definen los patrones regex para identificar cada tipo de PII. **Nota:** Esta es la parte más compleja del proceso, ya que crear expresiones regulares precisas y completas requiere considerar múltiples variaciones de formato para cada tipo de dato.

In [3]:
patterns = {
    'EMAIL':      r'[a-zA-Z0-9+._%-]+@[a-zA-Z0-9._%-]+\.[a-zA-Z]{2,}',
    'PHONE':      r'\b(?:\+?[\d]{1,3}[-.\s]?)*\(?\d{2,4}\)?[-.\s]?\d{3,4}[-.\s]?\d{3,4}\b',
    'ID':         r'\b\d{6,10}\b',                             
    'DIRECCION':  r'\bCalle\s+[A-Z][a-z]+(?:\s+\d+)?\b',       
    'NAME':       r'\b[A-ZÁÉÍÓÚ][a-záéíóú]+(?:\s+[A-ZÁÉÍÓÚ][a-záéíóú]+)*\b'
}

## 4. Función de Ofuscación

Esta función aplica todos los patrones regex al texto de entrada y reemplaza la información identificada con etiquetas de redacción:

In [4]:
def obfuscate(text):
    if not isinstance(text, str):
        return text
    for label, pattern in patterns.items():
        text = re.sub(pattern, f'[{label}_REDACTED]', text)
    return text

## 5. Aplicación de la Ofuscación al Dataset

Aplicamos la función de ofuscación a la columna 'Texto' del dataset para generar una versión limpia:

In [5]:
df['Texto_limpio'] = df['Texto'].apply(obfuscate)

## 6. Guardado del Dataset Limpio

In [6]:
df.to_csv('pii_dataset_cleaned.csv', index=False)

## 7. Visualización de Resultados

Comparemos algunos ejemplos del texto original vs. el texto ofuscado:

In [7]:
print("=== EJEMPLOS DE OFUSCACIÓN ===\n")

for i in range(3):  
    print(f"--- Ejemplo {i+1} ---")
    print("ORIGINAL:")
    print(df.iloc[i]['Texto'])
    print("\nOFUSCADO:")
    print(df.iloc[i]['Texto_limpio'])
    print("\n" + "="*50 + "\n")

=== EJEMPLOS DE OFUSCACIÓN ===

--- Ejemplo 1 ---
ORIGINAL:
Mi nombre es Reyes Pinilla Rosado, puedes contactarme por correo electrónico en woliver@tena.es, o llamarme al +34967 400 699. Vivo en Calle Rambla de Nieves Dalmau, 126, Ávila. Mi número de identificación es 40955969R.

OFUSCADO:
[NAME_REDACTED] nombre es [NAME_REDACTED], puedes contactarme por correo electrónico en [EMAIL_REDACTED], o llamarme al +[PHONE_REDACTED]. [NAME_REDACTED] en [DIRECCION_REDACTED] de [NAME_REDACTED], 126, [NAME_REDACTED]. [NAME_REDACTED] número de identificación es 40955969R.


--- Ejemplo 2 ---
ORIGINAL:
Mi nombre es Evita Solsona Escobar, puedes contactarme por correo electrónico en nicolaspou@hernandez.com, o llamarme al 991740961. Vivo en Calle Vial Luís Martín, 131, Las Palmas. Mi número de identificación es 34970246V.

OFUSCADO:
[NAME_REDACTED] nombre es [NAME_REDACTED], puedes contactarme por correo electrónico en [EMAIL_REDACTED], o llamarme al [PHONE_REDACTED]. [NAME_REDACTED] en [DIRECCION_R

## 8. Discusión Final

**¿Limitantes usando solo regex?**  
Sí. Las expresiones regulares son rígidas y no capturan todos los formatos de PII. A veces dan falsos positivos (p.ej., un número que no es teléfono) o negativos (p.ej., un DPI con letra).

**Posibles mejoras**  
- Usar NLP (NER) con bibliotecas como spaCy o transformers para entender el contexto.  
- Entrenar modelos de machine learning que complementen a las regex.  
- Implementar un enfoque híbrido: regex como filtro inicial y ML/NLP para validar.

**Implicaciones éticas**  
Si falla la ofuscación, se expone información sensible de pacientes (DNI, direcciones, correos), lo que puede violar HIPAA/GDPR, generar multas y dañar la confianza médico‑paciente. Por eso necesitamos soluciones robustas y claras sobre sus limitaciones.  
